In [12]:
import requests
import pandas as pd
import base_tools as tools
import numpy as np
from collections import defaultdict, OrderedDict
import operator
import json
from os.path import join as pjoin
import time
import os
import six
from pytablewriter import MarkdownTableWriter


https://api.opendota.com/api/matches/

In [13]:


filename = "archieved_match.json"
path_to_file = "D:\\Pro\\Py-DOta\\Dumped_Json\\archieved_match.json"
with open(path_to_file,'r') as f:
    data = json.load(f)
archieved_match = data

In [14]:
def post(df):
    df = df.groupby(df.columns, axis=1).sum()
    df['AVG_FS'] = df.apply(lambda row:row[0]/row[2],axis=1)
    df = df.sort_values(by=['AVG_FS'],ascending=False)
    df.columns = ['FS','Region','Matches','AVG_FS']
    return df


In [15]:
def get_fs_for_one_match(match_id):
    player_table = {}
    filename = str(match_id) + ".json"
    path_to_file = pjoin("D:\\Pro","Py-DOta","Dumped_Json",filename)
    if match_id in archieved_match:

        with open(path_to_file,'r') as f:
            details = json.load(f)
            f.close()
    else:
        target = "https://api.opendota.com/api/matches/" + str(match_id) 
        r = requests.get(url=target)
        time.sleep(2)
        details = r.json()
        with open(path_to_file,'w+') as f:
            json.dump(details,f)
            f.close()
        archieved_match.append(match_id)
        path_to_file = "D:\\Pro\\Py-DOta\\Dumped_Json\\archieved_match.json"
        with open(path_to_file,'w') as f:
            json.dump(archieved_match,f)
            f.close()
        
    try:

        fb_time = details['first_blood_time']
        radiant_kills = details['radiant_score']
        dire_kills = details['dire_score']
        #print(details.keys())
        try:
            region = details['region']
        except:
            region = 12

        
        for player in details['players']:
            #player_stats = []
            kills = player['kills']
            deaths = player['deaths']
            assists = player['assists']
            try:
                first_kill_time = player['kills_log'][0]['time']
            except:
                first_kill_time = -100
            if first_kill_time == fb_time:
                fb = 1
            else:
                fb = 0
            creep_scores = player['denies'] + player['last_hits']
            wards = player['obs_placed'] + player['sen_placed']
            camp = player['camps_stacked']
            stuns = player['stuns']
            gpm = player['gold_per_min']
            runes = player['rune_pickups']
            tower_kill = player['tower_kills']
            if player['isRadiant']:
                participant = (kills+assists)/radiant_kills
            else:
                participant = (kills+assists)/dire_kills
            roshan_kills = player['roshan_kills']
            player_fs = kills *0.3 - 0.3* deaths + 4.0* fb+ 0.003 * creep_scores+ 0.5*wards+0.5*camp+0.05*stuns+0.002*gpm+ 0.25*runes+tower_kill+3*participant+roshan_kills
            player_table[player['name']] = [player_fs,player['lane_role'],player['lh_t'][9],region,player['account_id']]
    except Exception as e:
        player_table[match_id] = str(e)
    return player_table

        

In [16]:
def get_fs_for_one_league(match_list,qualifier = False):
    all_table = {}
    mid_table = {}
    support_table = {}
    core_table = {}
    Error_ID = {}
    for match_id in match_list:

        player_table = get_fs_for_one_match(match_id)
        if len(player_table) == 1:
            k = list(player_table.keys())
            v = list(player_table.values())
            Error_ID[k[0]] = v[0]
            continue

        for k, v in player_table.items():
            if k in all_table:
                all_table[k][0] = all_table[k][0] + v[0] # total FS
                all_table[k][1].append(v[1]) # positions played
                all_table[k][2] = (all_table[k][2] + v[2])/2 # average lh@10mins
                all_table[k][4] = all_table[k][4]+ 1 # total matches played
            else:
                all_table[k] = [v[0],[v[1]],v[2],v[3],1,v[4]]
    for player,stats in all_table.items():
        role_list = stats[1]
        role_list.sort(reverse = True)
        d = defaultdict(int)
        for i in role_list:
            d[i] += 1
        (role,role_times) = max(d.items(),key=lambda x: x[1])
        if player == None:
            player = stats[5]
        if player == 'JAbz':
            print(role_list)
        lh_10_min = stats[2]
        if role == 2 and lh_10_min > 20:
            if qualifier:
                mid_table[player] = [stats[0],stats[3],stats[4]]
            else:
                mid_table[player] = [stats[0],stats[3],stats[4]]

                
        else:
            # if a plyaer have an average lh at 10 mins > 20, the player is regarded as a core.
            try:
                if lh_10_min > 20:
                    if qualifier:
                        core_table[player] = [stats[0],stats[3],stats[4]]
                    else:
                        core_table[player] = [stats[0],stats[3],stats[4]]
                else:
                    if qualifier:
                        support_table[player] = [stats[0],stats[3],stats[4]]
                    else:
                        support_table[player] = [stats[0],stats[3],stats[4]]
            except:
                print(lh_10_min)
    return all_table, mid_table,support_table, core_table,Error_ID

                

In [17]:
table, qualifier,main_event = tools.differed_id("Paris Major")


In [28]:
leagues = ["PIT Minor","DreamLeague S10","DreamLeague S11","SL-i Minor","Kuala Lumpur Major","Chongqing Major","Paris Major","Bucharest Minor"]

In [19]:
leagues = ["PIT Minor"]

In [20]:
all_table, mid_table, support_table, core_table,Error_ID = get_fs_for_one_league(qualifier,False)
Error_ID

[3, 3, 3, 3, 2, 2, 2, 1, 1]


{'4598714044': "unsupported operand type(s) for +: 'NoneType' and 'NoneType'",
 '4583318221': 'division by zero',
 '4581422908': "unsupported operand type(s) for +: 'NoneType' and 'NoneType'",
 '4581417620': "unsupported operand type(s) for +: 'NoneType' and 'NoneType'"}

In [30]:
mid_df_list = []
sup_df_list = []
core_df_list = []
mid_df_list2 = []
sup_df_list2 = []
core_df_list2 = []
cols = ['FS','Region','Matches','ID']
for league in leagues:
    table, qualifier,main_event = tools.differed_id(league)
    all_table, mid_table, support_table, core_table,Error_ID = get_fs_for_one_league(qualifier,False)
    df = pd.DataFrame(mid_table)
    df = df.T
    mid_df_list.append(df)
    df2 = pd.DataFrame(core_table)
    df2 = df2.T    
    core_df_list.append(df2)
    df3 = pd.DataFrame(support_table)
    df3 = df3.T  
    sup_df_list.append(df3)
    all_table, mid_table, support_table, core_table,Error_ID = get_fs_for_one_league(main_event,False)
    df = pd.DataFrame(mid_table)
    df = df.T
    mid_df_list2.append(df)
    df2 = pd.DataFrame(core_table)
    df2 = df2.T    
    core_df_list2.append(df2)
    df3 = pd.DataFrame(support_table)
    df3 = df3.T  
    sup_df_list2.append(df3)
mid_df = post(pd.concat(mid_df_list,axis = 1,sort=True))
mid_df.to_excel("mid_df_qu.xlsx",encoding='utf_8_sig')
core_df = post(pd.concat(core_df_list,axis = 1,sort=True))
core_df.to_excel("core_df_qu.xlsx",encoding='utf_8_sig')
sup_df = post(pd.concat(sup_df_list,axis = 1,sort=True))
sup_df.to_excel("sup_df_qu.xlsx",encoding='utf_8_sig')
mid_df2 = post(pd.concat(mid_df_list2,axis = 1,sort=True))
mid_df2.to_excel("mid_df_main.xlsx",encoding='utf_8_sig')
core_df2 = post(pd.concat(core_df_list2,axis = 1,sort=True))
core_df2.to_excel("core_df_main.xlsx",encoding='utf_8_sig')
sup_df2 = post(pd.concat(sup_df_list2,axis = 1,sort=True))
sup_df2.to_excel("sup_df_main.xlsx",encoding='utf_8_sig')


[4, 3, 3, 2, 2, 2, 1]
v
[3, 3, 3, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[3, 3, 3, 3, 2, 2, 1, 1, 1, 1, 1]
[3, 3, 3, 3, 3, 3, 2, 2, 2, 1, 1, 1, 1]
[3, 3, 2, 2, 2, 1, 1, 1, 1]
[3, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1]
[3, 3, 3, 3, 2, 2, 2, 1, 1]


In [22]:
df = pd.read_excel("mid_df_main.xlsx")
writer = MarkdownTableWriter()
writer.from_dataframe(df)
writer.write_table()
df

| FS  |Region|Matches|AVG_FS|
|----:|-----:|------:|-----:|
|246.7|     3|     20|12.335|
|157.7|     3|     13|12.134|
|223.7|     3|     19|11.775|
|195.6|     3|     17|11.507|
|124.1|     3|     11|11.284|
|148.1|     3|     14|10.578|
|144.8|     3|     14|10.341|
|118.6|     3|     12| 9.879|


,FS,Region,Matches,AVG_FS
Fata,246.698020,3,20,12.334901
_Mikoto_,157.737942,3,13,12.133688
430,223.730634,3,19,11.775297
qojqva,195.615552,3,17,11.506797
Setsu,124.123239,3,11,11.283931
Chris Luck,148.091411,3,14,10.577958
CCnC,144.771151,3,14,10.340796
Afoninje,118.551877,3,12,9.879323


In [23]:
df = pd.read_excel("core_df_main.xlsx")
writer = MarkdownTableWriter()
writer.from_dataframe(df)
writer.write_table()

|  FS  |Region|Matches|AVG_FS|
|-----:|-----:|------:|-----:|
|261.66|     3|     20|13.083|
|167.02|     3|     13|12.848|
|243.18|     3|     19|12.799|
|140.66|     3|     11|12.787|
|217.24|     3|     17|12.779|
|157.17|     3|     14|11.226|
|177.93|     3|     17|10.466|
|140.41|     3|     14|10.029|
|163.97|     3|     17| 9.645|
|182.41|     3|     19| 9.601|
|114.01|     3|     12| 9.501|
|126.60|     3|     14| 9.043|
|125.66|     3|     14| 8.976|
|112.05|     3|     13| 8.619|
|166.33|     3|     20| 8.316|
| 81.88|     3|     11| 7.444|
| 85.07|     3|     12| 7.090|
| 98.65|     3|     14| 7.046|


In [24]:
df3 = pd.concat([df,df2],axis = 1,sort=True)
df3.to_excel("df3.xlsx")

In [25]:

mid_df3 = mid_df2.groupby(mid_df2.columns, axis=1).sum()
mid_df3['AVG_FS'] = mid_df3.apply(lambda row:row[0]/row[2],axis=1)
mid_df3 = mid_df3.sort_values(by=['AVG_FS'],ascending=False)

In [26]:
mid_df3.


SyntaxError: invalid syntax (<ipython-input-26-43315555b89e>, line 1)

In [ ]:
new_list = []
for df in mid_df_list2:
    if not df.empty:
        new_list.append(df)

mid_df_list2 = new_list
mid_df2 = pd.concat(mid_df_list2,axis = 1,sort=True)

In [ ]:
mid_df_list2

In [27]:
sup_df2

,FS,Region,Matches,AVG_FS
y`,575.130640,3.0,19.0,30.270034
iNSaNia,495.970285,3.0,17.0,29.174723
Kitrak,360.105411,3.0,14.0,25.721815
Fng,306.741663,3.0,12.0,25.561805
Peterpandam,479.058365,3.0,20.0,23.952918
Khezcute,302.923531,3.0,13.0,23.301810
Yヤdドmミ,307.712720,3.0,14.0,21.979480
- ah fu -,226.716249,3.0,11.0,20.610568
XinQ,256.732425,3.0,19.0,13.512233
Saksa,267.079157,3.0,20.0,13.353958


In [ ]:

target = "https://api.opendota.com/api/players/201594424"
r = requests.get(url=target)
details = r.json()
details


In [31]:
mid_df = post(pd.concat(mid_df_list,axis = 1,sort=True))
mid_df.to_excel("mid_df_qu.xlsx",encoding='utf_8_sig')
core_df = post(pd.concat(core_df_list,axis = 1,sort=True))
core_df.to_excel("core_df_qu.xlsx",encoding='utf_8_sig')
sup_df = post(pd.concat(sup_df_list,axis = 1,sort=True))
sup_df.to_excel("sup_df_qu.xlsx",encoding='utf_8_sig')
mid_df2 = post(pd.concat(mid_df_list2,axis = 1,sort=True))
mid_df2.to_excel("mid_df_main.xlsx",encoding='utf_8_sig')
core_df2 = post(pd.concat(core_df_list2,axis = 1,sort=True))
core_df2.to_excel("core_df_main.xlsx",encoding='utf_8_sig')
sup_df2 = post(pd.concat(sup_df_list2,axis = 1,sort=True))
sup_df2.to_excel("sup_df_main.xlsx",encoding='utf_8_sig')